In [ ]:
""" 
why do I add to it? why not just a completly new dataset? Data leakage can be an argument
then just taking 17M (more than the 14384832 needed rows, since need to account for NAN's) rows from the complete_responseid_country dataset! """

# do the same preprocessing for the complete_responseid_country dataset as for the mme_dataset, so also responseID change and removing random from typestrict and removing columns

"DOING THE SAME AS FOR THE MME_DATASET, BUT THEN FOR ONLY COUNTRIES *NOT* IN THE WESTERN LIST"

In [17]:
import pandas as pd

# create a dataframe with only the UserCountry3 column from the 71M SharedResponses.csv file
reader_country = pd.read_csv('SharedResponses.csv', usecols=['UserCountry3', 'ResponseID'])


In [ ]:
# List of countries considered 'Western' from the UN

countries = ["AND",  "AUS", "AUT", "BEL", "CAN", "CHE", "DNK", "DEU", "ESP", "FIN",  "FRA",  "GBR",  "GRC",  "IRL", 
            "ISR", "ISL", "ITA", "LIE", "LUX", "MLT", "MCO", "NLD", "NOR", "NZL", "PRT", "SMR", "SWE", "TUR", "USA" ]

# Andorra, Australia, Austria, Belgium, Canada, Switzerland, Denmark, Germany, Spain, Finland, France, United Kingdom, Greece, Ireland, 
# Israel, Iceland, Italy, Liechtenstein, Luxembourg, Malta, Monaco, Netherlands, Norway, New Zealand, Portugal, San Marino, Sweden, Turkey, USA

In [56]:
# Filter the rows where UserCountryID is NOT the list of countries that are considered Western

non_western = reader_country[~reader_country['UserCountry3'].isin(countries)]


In [58]:
non_western.shape

(21837017, 2)

In [59]:
# only keep complete scenarios of 2

# Count how many times each ResponseID appears
response_counts = non_western['ResponseID'].value_counts()

# Get the ResponseIDs that appear exactly twice
ids_to_keep = response_counts[response_counts == 2].index

# Filter the rows where ResponseID is in the list of ids_to_keep
complete_responseid_country = non_western[non_western['ResponseID'].isin(ids_to_keep)]


In [60]:
print(complete_responseid_country.value_counts().min())
print(complete_responseid_country.value_counts().max())

2
2


In [61]:
complete_responseid_country.shape

(21084418, 2)

In [34]:
# checking the NAN's in UserCountry3 in the complete_responseid_country dataset and removing them
print(complete_responseid_country['UserCountry3'].isna().sum())

0


In [62]:
# removing the NAN's
complete_responseid_country = complete_responseid_country.dropna(subset=['UserCountry3'])
print(complete_responseid_country['UserCountry3'].isna().sum())
print(complete_responseid_country.shape)

0
(20439550, 2)


In [63]:
# Drop duplicates based on 'ResponseID' and keep the first occurrence
complete_responseid_country = complete_responseid_country.drop_duplicates(subset=['ResponseID'], keep='first')

In [64]:
country_subset = complete_responseid_country.tail(8500000)  # want 17M rows (with accounting for NAn's and deleting rows with 'random' (around 10%)), so 17M / 2 = 8.5M ResponseID's necessary

# taking the last 8,5 Million, as I assume the data in the set is random enough to not have a bias (like country) in the first 8,5M rows
# furthermore, this ensures new data, and not the same responses that were in the MME_total dataset (idk if this is important!!!!)

In [65]:
# checking if they are all unique

country_subset.nunique()

ResponseID      8500000
UserCountry3        202
dtype: int64

In [66]:
# transform to a list to feed to the for loop that will extract all the corresponding columns from the SharedResponses.csv file

country_sub_list = country_subset['ResponseID'].tolist()
# check if it went okay
print(len(country_sub_list))     # should be 8500000

8500000


In [67]:
subset_country = pd.DataFrame()

In [69]:
# reading SharedResponses to extract the rows with ResponseID's in country_sub_list
# this will result in a dataframe 'subset' that contains around 17 million rows (8,5 * 2)

chunk_size = 500_000
reader = pd.read_csv('SharedResponses.csv', chunksize=chunk_size, dtype=str, low_memory=False)

for i, chunk in enumerate(reader):
    
    print(f"Processing chunk {i+1}")

    # Filter rows where ResponseID is in reader_subset
    subset_chunk = chunk[chunk['ResponseID'].isin(country_sub_list)]

    # Append filtered chunk to empty df
    subset_country = pd.concat([subset_country, subset_chunk], ignore_index=True)

    print(f"Finished processing chunk {i+1}")

print("All chunks have been processed and combined.")

Processing chunk 1
Finished processing chunk 1
Processing chunk 2
Finished processing chunk 2
Processing chunk 3
Finished processing chunk 3
Processing chunk 4
Finished processing chunk 4
Processing chunk 5
Finished processing chunk 5
Processing chunk 6
Finished processing chunk 6
Processing chunk 7
Finished processing chunk 7
Processing chunk 8
Finished processing chunk 8
Processing chunk 9
Finished processing chunk 9
Processing chunk 10
Finished processing chunk 10
Processing chunk 11
Finished processing chunk 11
Processing chunk 12
Finished processing chunk 12
Processing chunk 13
Finished processing chunk 13
Processing chunk 14
Finished processing chunk 14
Processing chunk 15
Finished processing chunk 15
Processing chunk 16
Finished processing chunk 16
Processing chunk 17
Finished processing chunk 17
Processing chunk 18
Finished processing chunk 18
Processing chunk 19
Finished processing chunk 19
Processing chunk 20
Finished processing chunk 20
Processing chunk 21
Finished processin

In [71]:
subset_country.shape  # 17M rows, 41 columns

(17000000, 41)

In [80]:
# checking to see if the UserCountry3 column only contains countries not in the Western list
print(subset_country['UserCountry3'].isin(countries).value_counts())

# should be False for all rows (17M)

UserCountry3
False    17000000
Name: count, dtype: int64


In [81]:
subset_country_csv = subset_country.to_csv('subset_country.csv', index=False)

In [2]:
import pandas as pd
df_country = pd.read_csv('subset_country.csv')

C:\Users\esmku\AppData\Local\Temp\ipykernel_24844\2704394548.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_country = pd.read_csv('subset_country.csv')


df_country.head

In [3]:
df_country.shape

(17000000, 41)

In [4]:
df_country['ScenarioTypeStrict'].value_counts()

ScenarioTypeStrict
Utilitarian      3051804
Species          2999718
Fitness          2998592
Age              2996212
Gender           2995012
Random           1495412
Social Status     463250
Name: count, dtype: int64

In [5]:
# removing all rows with 'random' in the ScenarioTypeStrict column

df_country = df_country[df_country['ScenarioTypeStrict'] != 'Random']

In [6]:
df_country.shape # should be 17000000 - 1495412 = 15504588

(15504588, 41)

In [7]:
df_country['ScenarioTypeStrict'].value_counts()

ScenarioTypeStrict
Utilitarian      3051804
Species          2999718
Fitness          2998592
Age              2996212
Gender           2995012
Social Status     463250
Name: count, dtype: int64

In [9]:
df_country.isna().sum()

ResponseID                       0
ExtendedSessionID                0
UserID                         822
ScenarioOrder                    0
Intervention                     0
PedPed                           0
Barrier                          0
CrossingSignal                   0
AttributeLevel                   0
ScenarioTypeStrict               0
ScenarioType                     2
DefaultChoice               492908
NonDefaultChoice            492908
DefaultChoiceIsOmission     492908
NumberOfCharacters               2
DiffNumberOFCharacters           2
Saved                            0
Template                   1855572
DescriptionShown           1855572
LeftHand                   1855572
UserCountry3                     0
Man                              1
Woman                            2
Pregnant                         2
Stroller                         2
OldMan                           2
OldWoman                         2
Boy                              2
Girl                

In [10]:
# deleting rows with NaN's in the UserID column

df_country = df_country.dropna(subset=['UserID'])

In [11]:
df_country.shape # should be 15504588 - 822 = 15503766

(15503766, 41)

In [17]:
# rows that are needed to make dataset 2% llm and 98% human
total_rows_dataset = 293568 / 2 * 98
print(total_rows_dataset)    
rows_to_delete = df_country.shape[0] - total_rows_dataset
print(rows_to_delete) 

# need to delete 1118934 rows 

14384832.0
1118934.0


In [18]:
# randomly delete 1118934 / 2 = 559467 unique UserIDs (is 1118934 rows), to ensure 2% of the dataset is LLMs and 98% humans

# Getting unique UserIDs
Response_unique = df_country['ResponseID'].unique()
print(len(Response_unique))  # should be 15503766/2 = 7751883

# Selecting 559467 UserIDs from the unique set
Response_delete = pd.Series(Response_unique).sample(n=559467, random_state=42)


7751883


In [19]:
df_country_98 = df_country[~df_country['ResponseID'].isin(Response_delete)]

In [20]:
# checking shape of df_filtered

df_country_98.shape   # should be 15503310 - 1118934 = 14384832

(14384832, 41)

In [21]:
# Changing the responseID

# Define the starting point for the new ResponseID
starting_id = 146784        # cause this is where llm rows ended

# Get the total number of rows
num_rows = len(df_country_98)

# Create a new column where every two rows have the same ResponseID
df_country_98['ResponseID'] = [f'res_{starting_id + i // 2:08d}' for i in range(num_rows)]

C:\Users\esmku\AppData\Local\Temp\ipykernel_24844\2329753221.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country_98['ResponseID'] = [f'res_{starting_id + i // 2:08d}' for i in range(num_rows)]


In [22]:
# ResponseID starts with res_00146784
df_country_98.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
1,res_00146784,477341325_9658440103616898.0,9.658440e+15,10,0,0,0,0,Hoomans,Species,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,res_00146784,-1933553294_3514601197273468.0,3.514601e+15,11,0,1,0,0,Pets,Species,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
5,res_00146785,-1906528630_1615619368,1.615619e+09,7,0,0,0,0,Fit,Fitness,...,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
6,res_00146785,1900197736_3778157516.0,3.778158e+09,10,0,1,0,0,Less,Utilitarian,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,res_00146786,1256110879_1407997320198901.0,1.407997e+15,10,0,0,0,2,Male,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# ResponseID ends with res_00146784 + (14384832/2) = res_07339200
df_country_98.tail()

# somehow the last responseID is res_07339199, but it is correct I think

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
16999994,res_07339197,1135690765_653858658,6.538587e+08,13,1,0,0,0,Fat,Fitness,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16999995,res_07339198,625350849_8584219519940754.0,8.584220e+15,6,1,1,0,1,Pets,Species,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
16999997,res_07339198,434100736_1451013350,1.451013e+09,1,1,1,0,0,More,Utilitarian,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16999998,res_07339199,-1815706184_742769765189808.0,7.427698e+14,5,1,1,0,0,Young,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16999999,res_07339199,-1475685997_4499486314855814.0,4.499486e+15,4,1,1,0,2,Fit,Fitness,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [24]:
df_country_98.columns

Index(['ResponseID', 'ExtendedSessionID', 'UserID', 'ScenarioOrder',
       'Intervention', 'PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel',
       'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters',
       'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown',
       'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
       'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
       'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
       'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
       'Cat'],
      dtype='object')

In [25]:
# already deleting some columns to ensure it fits in memory
# deleting the columns that are not necessary for the modelling: ExtendedSessionID, DefaultChoice, NonDefaultChoice, DefaultChoiceIsOmission, Template

df_country_clean = df_country_98.drop(columns=['ExtendedSessionID', 'DefaultChoice', 'NonDefaultChoice', 'DefaultChoiceIsOmission', 'Template'])

In [26]:
df_country_clean.shape # should be 14367864 rows and 41-5 = 36 columns

(14384832, 36)

In [27]:
# saving this pre-preprocessed non-western dataset to a csv file

df_country_clean.to_csv('non_western_dataset.csv', index=False)